<a href="https://colab.research.google.com/github/Thushan97/CURE/blob/master/cure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/Thushan97/CURE.git

Cloning into 'CURE'...
remote: Enumerating objects: 1031, done.
remote: Counting objects: 100% (359/359), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 1031 (delta 69), reused 289 (delta 29), pack-reused 672
Receiving objects: 100% (1031/1031), 79.63 MiB | 7.35 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [4]:
!mv /content/CURE/* /content/

In [5]:
# pretrain model download
!wget "https://zenodo.org/record/7030145/files/models.tar.xz?download=1" -c -O 'models.tar.xz'
!mkdir /content/data/models
!tar -xf models.tar.xz
!mv /content/models/* /content/data/models/


--2022-09-20 14:17:52--  https://zenodo.org/record/7030145/files/models.tar.xz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1911937724 (1.8G) [application/octet-stream]
Saving to: ‘models.tar.xz’

models.tar.xz       100%[===================>]   1.78G  15.4MB/s    in 2m 33s  

2022-09-20 14:20:28 (11.9 MB/s) - ‘models.tar.xz’ saved [1911937724/1911937724]



In [6]:
!pip install transformers==2.10.0 subword-nmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 660 kB 31.6 MB/s 
     |████████████████████████████████| 880 kB 55.3 MB/s 
     |████████████████████████████████| 5.6 MB 48.4 MB/s 
     |████████████████████████████████| 1.3 MB 43.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=470f10a964bab34499a12436ce946ff31d354af82e1ac1969166c152e732c7ba
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


---

Run gpt_conut_trainer.py file

In [14]:
import os
import sys
import json
import time
import codecs
import random
import numpy as np
import torch
import torch.nn as nn
from transformers import OpenAIGPTLMHeadModel

GPT_CONUT_TRAINER_DIR = os.path.abspath('/content')#os.path.abspath(__file__)[: os.path.abspath(__file__).rindex('/') + 1]

In [15]:
from src.models.gpt_conut import GPTCoNuTModel
from src.dataloader.dictionary import Dictionary
from src.dataloader.gpt_conut_data_loader import GPTCoNuTDataLoader

# New Section

In [9]:
# print(f'CUDA GPU availible : {torch.cuda.is_available()}')
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
# class GPTCoNuTTrainer():
#     def __init__(self, train_loader, valid_loader, dictionary, gpt_file):
#         gpt_loaded = torch.load(gpt_file)
#         config = gpt_loaded['config']
#         gpt_model = OpenAIGPTLMHeadModel(config).cuda()
#         gpt_model.load_state_dict(gpt_loaded['model'])

#         self.train_loader = train_loader
#         self.valid_loader = valid_loader
#         self.dictionary = dictionary

#         self.batch_size = 12
#         self.load_size = 1200   # load 1200 samples from training data every time

#         self.gpt_model = gpt_model
#         self.model = None
#         self.hyper_parameter = {}
#         self.optimizer = None
#         self.current_train_step = 0
#         self.val_loss = {}

#     def shuffle_dataset(self):
#         indices = [i for i in range(len(self.train_loader.dataset))]
#         random.shuffle(indices)
#         return indices

#     def train_step(self, samples):
#         self.model.train()
#         self.current_train_step += 1
#         self.optimizer.zero_grad()

#         batch = self.train_loader.dataset.collater(samples)
#         if torch.cuda.is_available():
#             outputs = self.model(
#                 batch['net_input']['src_tokens'].cuda(),
#                 batch['net_input']['src_with_prev_context'].cuda(),
#                 batch['net_input']['ctx_tokens'].cuda(),
#                 prev_tokens_index=batch['target_index'].cuda(),
#                 prev_tokens_with_context=batch['target_with_prev_context'].cuda(),
#                 labels=batch['target'].cuda(),
#             )
#         else:
#             outputs = self.model(
#                 batch['net_input']['src_tokens'],
#                 batch['net_input']['src_with_prev_context'],
#                 batch['net_input']['ctx_tokens'],
#                 prev_tokens_index=batch['target_index'],
#                 prev_tokens_with_context=batch['target_with_prev_context'],
#                 labels=batch['target'],
#             )
#         logits, avg_attn_scores, apr_loss, lm_loss = outputs[:4]
#         loss = apr_loss + 0.3 * lm_loss
#         loss.mean().backward()
#         nn.utils.clip_grad_norm_(self.model.parameters(), 0.5, norm_type=2)
#         self.optimizer.step()
#         return loss.mean().item(), apr_loss.mean().item(), lm_loss.mean().item()

#     def valid_step(self, samples):
#         self.model.eval()
#         batch = self.valid_loader.dataset.collater(samples)
#         outputs = self.model(
#             batch['net_input']['src_tokens'].cuda(),
#             batch['net_input']['src_with_prev_context'].cuda(),
#             batch['net_input']['ctx_tokens'].cuda(),
#             prev_tokens_index=batch['target_index'].cuda(),
#             prev_tokens_with_context=batch['target_with_prev_context'].cuda(),
#             labels=batch['target'].cuda(),
#         )
#         logits, avg_attn_scores, apr_loss, lm_loss = outputs[:4]
#         loss = apr_loss + 0.3 * lm_loss
#         return loss.mean().item(), apr_loss.mean().item(), lm_loss.mean().item(), logits

#     def validate_and_save(self, model_id, save_dir):
#         oom = 0
#         with torch.no_grad():
#             val_loss, val_fconv_loss, val_lm_loss = [], [], []
#             for i in range(0, self.valid_loader.total_size, self.batch_size):
#                 samples = [self.valid_loader.dataset[j]
#                            for j in range(i, min(len(self.valid_loader.dataset), i + self.batch_size))]
#                 try:
#                     loss, fconv_loss, lm_loss, logits = self.valid_step(samples)
#                     val_loss.append(float(loss))
#                     val_fconv_loss.append(float(fconv_loss))
#                     val_lm_loss.append(float(lm_loss))
#                 except Exception as e:
#                     oom += 1

#             info = 'val loss:{}, val apr_loss:{}, val lm_loss:{}, val ppl:{}, oom:{}'.format(
#                 round(float(np.mean(val_loss)), 6),
#                 round(float(np.mean(val_fconv_loss)), 6),
#                 round(float(np.mean(val_lm_loss)), 6),
#                 round(float(np.exp(np.mean(val_loss))), 6),
#                 oom
#             )
#             print(info)

#             val_loss = np.mean(val_fconv_loss)
#             checkpoint = {
#                 'model': self.model.state_dict(),
#                 'optimizer': self.optimizer.state_dict(),
#                 'current_step': self.current_train_step,
#                 'config': self.model.config(),
#                 'val_loss': val_loss,
#             }
#             torch.save(checkpoint, save_dir + 'gpt_conut_' + str(model_id) + '.pt')
#             self.val_loss[model_id] = {
#                 'val_loss': val_loss,
#                 'hyper-parameter': str(self.hyper_parameter),
#             }

#         return val_loss

#     def train(self, model_id, epochs, hyper_parameter, save_dir):
#         self.hyper_parameter = hyper_parameter
#         self.model = GPTCoNuTModel(
#             self.dictionary, embed_dim=384, max_positions=1024,
#             src_encoder_convolutions=self.hyper_parameter['src_encoder_convolutions'],
#             ctx_encoder_convolutions=self.hyper_parameter['ctx_encoder_convolutions'],
#             decoder_convolutions=self.hyper_parameter['decoder_convolutions'],
#             dropout=self.hyper_parameter['dropout'], embed_model=self.gpt_model,
#         ).cuda()
#         self.optimizer = torch.optim.Adam(self.model.parameters(), lr=6.25e-5)
#         # self.model = nn.DataParallel(self.model, device_ids=device_ids)
        
#         self.valid_loader.load_data(0, self.valid_loader.total_size)
#         for epoch in range(epochs):
#             start_time = time.time()
#             for i in range(0, self.train_loader.total_size, self.load_size):
#                 oom = 0
#                 self.train_loader.load_data(i, i + self.load_size)
#                 indices = self.shuffle_dataset()
#                 train_loss, train_apr_loss, train_lm_loss = [], [], []

#                 start, end = 0, 0
#                 samples = []
#                 max_src, max_ctx, max_tgt = 0, 0, 0
#                 while end < len(self.train_loader.dataset):
#                     sample = self.train_loader.dataset[indices[end]]
#                     if max_ctx + len(sample['target']) >= 1023 \
#                             or max_tgt + len(sample['prev_context']) >= 1023 \
#                             or max_ctx + len(sample['source']) >= 1023 \
#                             or max_src + len(sample['prev_context']) >= 1023 \
#                             or end - start == self.batch_size:
#                         try:
#                             loss, apr_loss, lm_loss = self.train_step(samples)
#                             train_loss.append(loss)
#                             train_apr_loss.append(apr_loss)
#                             train_lm_loss.append(lm_loss)
#                         except Exception as e:
#                             oom += 1

#                         start = end
#                         max_src, max_ctx, max_tgt = 0, 0, 0
#                         samples = []
#                         continue
#                     max_src = max(max_src, len(sample['source']))
#                     max_ctx = max(max_ctx, len(sample['prev_context']))
#                     max_tgt = max(max_tgt, len(sample['target']))
#                     end += 1
#                     samples.append(sample)
#                 if len(samples) > 0:
#                     try:
#                         loss, apr_loss, lm_loss = self.train_step(samples)
#                         train_loss.append(loss)
#                         train_apr_loss.append(apr_loss)
#                         train_lm_loss.append(lm_loss)
#                     except Exception as e:
#                         oom += 1

#                 if (i // self.load_size) % 10 == 0:
#                     info = 'epoch:{}, load data:{}, lr:{}, loss:{}, apr_loss:{}, lm_loss:{}, time:{}s, oom:{}'.\
#                         format(epoch + 1, i + self.load_size,
#                                round(self.optimizer.param_groups[0]['lr'], 10),
#                                round(float(np.mean(train_loss)), 6),
#                                round(float(np.mean(train_apr_loss)), 6),
#                                round(float(np.mean(train_lm_loss)), 6),
#                                int(time.time() - start_time), oom
#                                )
#                     start_time = time.time()
#                     print(str(model_id) + ' ' + info)

#                 if (i // self.load_size) % 100 == 0:
#                     self.validate_and_save(model_id, save_dir)
#         self.validate_and_save(model_id, save_dir)

In [12]:
# if __name__ == '__main__':
#     device_ids = [0, 1, 2, 3]
#     os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
    
#     vocab_file = GPT_CONUT_TRAINER_DIR + '/data/vocabulary/vocabulary.txt'
#     train_file = GPT_CONUT_TRAINER_DIR + '/data/data/training_bpe.txt'
#     valid_file = GPT_CONUT_TRAINER_DIR + '/data/data/validation_bpe.txt'
#     gpt_file = GPT_CONUT_TRAINER_DIR + '/data/models/code_gpt.pt'

#     dictionary = Dictionary(vocab_file, min_cnt=0)
#     print('dictionary initialized, vocab size:{}'.format(len(dictionary)))

#     train_loader = GPTCoNuTDataLoader(train_file, dictionary)
#     valid_loader = GPTCoNuTDataLoader(valid_file, dictionary)
#     print('data loader initialized, train size:{}, validate size:{}'.
#           format(train_loader.total_size, valid_loader.total_size))

#     trainer = GPTCoNuTTrainer(train_loader, valid_loader, dictionary, gpt_file)

#     hyper_parameter = {
#         'src_encoder_convolutions': ((192, 5),) * 1,
#         'ctx_encoder_convolutions': ((384, 5),) * 1,
#         'decoder_convolutions': ((192, 5),) * 1,
#         'dropout': 0.1,
#     }
#     model_id = 1
#     epochs = 5
#     trainer.train(model_id, epochs, hyper_parameter, save_dir=GPT_CONUT_TRAINER_DIR + '/data/models/')

Run gpt_fconv_trainer.py file

In [16]:
import json
import os
import sys
import time
import codecs
import random
import numpy as np
import torch
import torch.nn as nn
from transformers import OpenAIGPTLMHeadModel

# GPT_FCONV_TRAINER_DIR = os.path.abspath(__file__)[: os.path.abspath(__file__).rindex('/') + 1]
GPT_FCONV_TRAINER_DIR = os.path.abspath('/content')

from src.models.gpt_fconv import GPTFConvModel
from src.dataloader.dictionary import Dictionary
from src.dataloader.gpt_fconv_data_loader import GPTFConvDataLoader


# class GPTFConvTrainer():
#     def __init__(self, train_loader, valid_loader, dictionary, gpt_file):
#         gpt_loaded = torch.load(gpt_file)
#         config = gpt_loaded['config']
#         gpt_model = OpenAIGPTLMHeadModel(config).cuda()
#         gpt_model.load_state_dict(gpt_loaded['model'])

#         self.train_loader = train_loader
#         self.valid_loader = valid_loader
#         self.dictionary = dictionary

#         self.batch_size = 12
#         self.load_size = 1200

#         self.gpt_model = gpt_model
#         self.model = None
#         self.hyper_parameter = {}
#         self.hyper_parameter_set = {'{}'}
#         self.optimizer = None
#         self.current_train_step = 0
#         self.val_loss = {}

#     def shuffle_dataset(self):
#         indices = [i for i in range(len(self.train_loader.dataset))]
#         random.shuffle(indices)
#         return indices

#     def train_step(self, samples):
#         self.model.train()
#         self.current_train_step += 1
#         self.optimizer.zero_grad()

#         batch = self.train_loader.dataset.collater(samples)
#         if torch.cuda.is_available():
#             outputs = self.model(
#                 batch['net_input']['src_tokens'].cuda(),
#                 batch['net_input']['src_with_prev_context'].cuda(),
#                 prev_tokens_index=batch['target_index'].cuda(),
#                 prev_tokens_with_context=batch['target_with_prev_context'].cuda(),
#                 labels=batch['target'].cuda(),
#             )

#         logits, avg_attn_scores, apr_loss, lm_loss = outputs[:4]
#         loss = apr_loss + 0.3 * lm_loss
#         loss.mean().backward()
#         nn.utils.clip_grad_norm_(self.model.parameters(), 0.5, norm_type=2)
#         self.optimizer.step()
#         return loss.mean().item(), apr_loss.mean().item(), lm_loss.mean().item()

#     def valid_step(self, samples):
#         self.model.eval()
#         batch = self.valid_loader.dataset.collater(samples)
#         outputs = self.model(
#             batch['net_input']['src_tokens'].cuda(),
#             batch['net_input']['src_with_prev_context'].cuda(),
#             prev_tokens_index=batch['target_index'].cuda(),
#             prev_tokens_with_context=batch['target_with_prev_context'].cuda(),
#             labels=batch['target'].cuda(),
#         )
#         logits, avg_attn_scores, apr_loss, lm_loss = outputs[:4]
#         loss = apr_loss + 0.3 * lm_loss
#         return loss.mean().item(), apr_loss.mean().item(), lm_loss.mean().item(), logits

#     def validate_and_save(self, model_id, save_dir):
#         oom = 0
#         with torch.no_grad():
#             val_loss, val_fconv_loss, val_lm_loss = [], [], []
#             for i in range(0, self.valid_loader.total_size, self.batch_size):
#                 samples = [self.valid_loader.dataset[j]
#                            for j in range(i, min(len(self.valid_loader.dataset), i + self.batch_size))]
#                 try:
#                     loss, fconv_loss, lm_loss, logits = self.valid_step(samples)
#                     val_loss.append(float(loss))
#                     val_fconv_loss.append(float(fconv_loss))
#                     val_lm_loss.append(float(lm_loss))
#                 except Exception as e:
#                     oom += 1

#             info = 'val loss:{}, val apr_loss:{}, val lm_loss:{}, val ppl:{}, oom:{}'.format(
#                 round(float(np.mean(val_loss)), 6),
#                 round(float(np.mean(val_fconv_loss)), 6),
#                 round(float(np.mean(val_lm_loss)), 6),
#                 round(float(np.exp(np.mean(val_loss))), 6),
#                 oom
#             )
#             print(info)

#             val_loss = np.mean(val_fconv_loss)
#             checkpoint = {
#                 'model': self.model.state_dict(),
#                 'optimizer': self.optimizer.state_dict(),
#                 'current_step': self.current_train_step,
#                 'config': self.model.config(),
#                 'val_loss': val_loss,
#             }
#             torch.save(checkpoint, save_dir + 'gpt_fconv_' + str(model_id) + '.pt')
#             self.val_loss[model_id] = {
#                 'val_loss': val_loss,
#                 'hyper-parameter': str(self.hyper_parameter),
#             }
#         return val_loss

#     def train(self, model_id, epochs, hyper_parameter, save_dir):
#         self.hyper_parameter = hyper_parameter
#         self.model = GPTFConvModel(
#                 self.dictionary, embed_dim=384, max_positions=1024,
#                 encoder_convolutions=self.hyper_parameter['encoder_convolutions'],
#                 decoder_convolutions=self.hyper_parameter['decoder_convolutions'],
#                 dropout=self.hyper_parameter['dropout'], embed_model=self.gpt_model,
#             ).cuda()
#         self.optimizer = torch.optim.Adam(self.model.parameters(), lr=6.25e-5)
#         # self.model = nn.DataParallel(self.model, device_ids=device_ids)
        
#         self.valid_loader.load_data(0, self.valid_loader.total_size)
#         for epoch in range(epochs):
#             start_time = time.time()
#             for i in range(0, self.train_loader.total_size, self.load_size):
#                 oom = 0
#                 self.train_loader.load_data(i, i + self.load_size)
#                 indices = self.shuffle_dataset()
#                 train_loss, train_apr_loss, train_lm_loss = [], [], []

#                 start, end = 0, 0
#                 samples = []
#                 max_src, max_ctx, max_tgt = 0, 0, 0
#                 while end < len(self.train_loader.dataset):
#                     sample = self.train_loader.dataset[indices[end]]
#                     if max_ctx + len(sample['target']) >= 1023 \
#                             or max_tgt + len(sample['prev_context']) >= 1023 \
#                             or max_ctx + len(sample['source']) >= 1023 \
#                             or max_src + len(sample['prev_context']) >= 1023 \
#                             or end - start == self.batch_size:
#                         try:
#                             loss, apr_loss, lm_loss = self.train_step(samples)
#                             train_loss.append(loss)
#                             train_apr_loss.append(apr_loss)
#                             train_lm_loss.append(lm_loss)
#                         except Exception as e:
#                             oom += 1

#                         start = end
#                         max_src, max_ctx, max_tgt = 0, 0, 0
#                         samples = []
#                         continue
#                     max_src = max(max_src, len(sample['source']))
#                     max_ctx = max(max_ctx, len(sample['prev_context']))
#                     max_tgt = max(max_tgt, len(sample['target']))
#                     end += 1
#                     samples.append(sample)
#                 if len(samples) > 0:
#                     try:
#                         loss, apr_loss, lm_loss = self.train_step(samples)
#                         train_loss.append(loss)
#                         train_apr_loss.append(apr_loss)
#                         train_lm_loss.append(lm_loss)
#                     except Exception as e:
#                         oom += 1

#                 if (i // self.load_size) % 10 == 0:
#                     info = 'epoch:{}, load data:{}, lr:{}, loss:{}, apr_loss:{}, lm_loss:{}, time:{}s, oom:{}'.\
#                         format(epoch + 1, i + self.load_size,
#                                round(self.optimizer.param_groups[0]['lr'], 10),
#                                round(float(np.mean(train_loss)), 6),
#                                round(float(np.mean(train_apr_loss)), 6),
#                                round(float(np.mean(train_lm_loss)), 6),
#                                int(time.time() - start_time), oom
#                                )
#                     start_time = time.time()
#                     print(str(model_id) + ' ' + info)

#                 if (i // self.load_size) % 100 == 0:
#                     self.validate_and_save(model_id, save_dir)
#         self.validate_and_save(model_id, save_dir)



In [18]:


# device_ids = [0, 1, 2, 3]
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
    
# vocab_file = GPT_FCONV_TRAINER_DIR + '/data/vocabulary/vocabulary.txt'
# train_file = GPT_FCONV_TRAINER_DIR + '/data/data/training_bpe.txt'
# valid_file = GPT_FCONV_TRAINER_DIR + '/data/data/validation_bpe.txt'
# gpt_file = GPT_FCONV_TRAINER_DIR + '/data/models/code_gpt.pt'

# dictionary = Dictionary(vocab_file, min_cnt=0)
# print('dictionary initialized, vocab size:{}'.format(len(dictionary)))

# train_loader = GPTFConvDataLoader(train_file, dictionary)
# valid_loader = GPTFConvDataLoader(valid_file, dictionary)
# print('data loader initialized, train size:{}, validate size:{}'.
#       format(train_loader.total_size, valid_loader.total_size))

# trainer = GPTFConvTrainer(train_loader, valid_loader, dictionary, gpt_file)

# hyper_parameter = {
#     'encoder_convolutions': ((192, 5),) * 1,
#     'decoder_convolutions': ((192, 5),) * 1,
#     'dropout': 0.1,
# }
# trainer.train(1, 2, hyper_parameter, save_dir=GPT_FCONV_TRAINER_DIR + '/data/models/')


Run generator.py file

In [19]:
import codecs
import torch
import sys
import os
from transformers import OpenAIGPTLMHeadModel

# GENERATOR_DIR = os.path.abspath(__file__)[: os.path.abspath(__file__).rindex('/') + 1]
GENERATOR_DIR = os.path.abspath('/content')
# sys.path.append(GENERATOR_DIR + '/models/')
# sys.path.append(GENERATOR_DIR + '/dataloader/')
# sys.path.append(GENERATOR_DIR + '/tester/')
from src.dataloader.gpt_conut_data_loader import GPTCoNuTDataLoader
from src.dataloader.gpt_fconv_data_loader import GPTFConvDataLoader
from src.dataloader.identifier_data_loader import IdentifierDataLoader
from src.dataloader.dictionary import Dictionary
from src.models.gpt_conut import GPTCoNuTModel
from src.models.gpt_fconv import GPTFConvModel
from src.tester.beamsearch import BeamSearch


class Generator():
    def __init__(self, model, model_type, dictionary, data_loader, beam_size=10):
        self.model = model
        self.dictionary = dictionary
        self.data_loader = data_loader
        self.beam_size = beam_size
        self.beamsearch = BeamSearch(model, model_type, dictionary, beam_size)
        print(self.model, beam_size)

    def generate(self, output_path):
        wp = codecs.open(output_path, 'w', 'utf-8')
        self.data_loader.load_data(0, self.data_loader.total_size)
        for i in range(self.data_loader.total_size):
            print(i, '/', self.data_loader.total_size)
            data = self.data_loader.dataset[i]
            if True:
                self.beamsearch.beam_size = self.beam_size
                sample = self.data_loader.dataset.collater([data])
                with torch.no_grad():
                    if isinstance(self.model, GPTCoNuTModel):
                        hypothesis = self.beamsearch.generate_gpt_conut(sample)
                    elif isinstance(self.model, GPTFConvModel):
                        hypothesis = self.beamsearch.generate_gpt_fconv(sample)
            # except Exception as e:
            #    print(e)
            #    continue
            id = str(sample['id'].item())
            wp.write('S-{}\t'.format(id))
            wp.write(self.dictionary.string(data['source']) + '\n')
            wp.write('T-{}\t'.format(id))
            wp.write(self.dictionary.string(data['target']) + '\n')
            for h in hypothesis:
                wp.write('H-{}\t{}\t'.format(id, str(h['final_score'])))
                wp.write(self.dictionary.string(h['hypo']) + '\n')
                wp.write('P-{}\t'.format(id))
                wp.write(' '.join(str(round(s.item(), 4)) for s in h['score']) + '\n')
        wp.close()


def generate_gpt_conut(vocab_file, model_file, input_file, identifier_txt_file, identifier_token_file, output_file, beam_size):
    dictionary = Dictionary(vocab_file, min_cnt=0)
    print(len(dictionary))
    loaded = torch.load(model_file, map_location='cpu')
    config = loaded['config']
    gpt_config = config['embed_model_config']
    gpt_config.attn_pdrop = 0
    gpt_config.embd_pdrop = 0
    gpt_config.resid_pdrop = 0
    gpt_model = OpenAIGPTLMHeadModel(gpt_config)
    model = GPTCoNuTModel(
        dictionary=dictionary, embed_dim=config['embed_dim'],
        max_positions=config['max_positions'],
        src_encoder_convolutions=config['src_encoder_convolutions'],
        ctx_encoder_convolutions=config['ctx_encoder_convolutions'],
        decoder_convolutions=config['decoder_convolutions'],
        dropout=0, embed_model=gpt_model,
    )

    model.load_state_dict(loaded['model'])
    identifier_loader = IdentifierDataLoader(
        dictionary, identifier_token_file, identifier_txt_file
    )
    data_loader = GPTCoNuTDataLoader(
        input_file, dictionary,
        identifier_loader=identifier_loader
    )
    generator = Generator(model, "conut", dictionary, data_loader, beam_size=beam_size)
    print('start generate')
    generator.generate(output_file)


def generate_gpt_fconv(vocab_file, model_file, input_file, identifier_txt_file, identifier_token_file, output_file, beam_size):
    dictionary = Dictionary(vocab_file, min_cnt=0)
    print(len(dictionary))
    loaded = torch.load(
        model_file, map_location='cpu'
    )
    config = loaded['config']
    gpt_config = config['embed_model_config']
    gpt_config.attn_pdrop = 0
    gpt_config.embd_pdrop = 0
    gpt_config.resid_pdrop = 0
    gpt_model = OpenAIGPTLMHeadModel(gpt_config)
    model = GPTFConvModel(
        dictionary=dictionary, embed_dim=config['embed_dim'],
        max_positions=config['max_positions'],
        encoder_convolutions=config['encoder_convolutions'],
        decoder_convolutions=config['decoder_convolutions'],
        dropout=0, embed_model=gpt_model,
    )
    model.load_state_dict(loaded['model'])
    identifier_loader = IdentifierDataLoader(
        dictionary, identifier_token_file, identifier_txt_file
    )
    data_loader = GPTFConvDataLoader(
        input_file, dictionary,
        identifier_loader=identifier_loader
    )
    generator = Generator(model, "fconv", dictionary, data_loader, beam_size=beam_size)
    print('start generate')
    generator.generate(output_file)


In [20]:
vocab_file = GENERATOR_DIR + '/data/vocabulary/vocabulary.txt'
input_file = GENERATOR_DIR + '/candidate_patches/QuixBugs/quixbugs_bpe.txt'
identifier_txt_file = GENERATOR_DIR + '/candidate_patches/QuixBugs/identifier.txt'
identifier_token_file = GENERATOR_DIR + '/candidate_patches/QuixBugs/identifier.tokens'
beam_size = 1000
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

model_file = GENERATOR_DIR + '/data/models/gpt_fconv_1.pt'
output_file = GENERATOR_DIR + '/data/patches/gpt_fconv_1.txt'
generate_gpt_fconv(vocab_file, model_file, input_file, identifier_txt_file, identifier_token_file, output_file, beam_size)

model_file = GENERATOR_DIR + '/data/models/gpt_conut_1.pt'
output_file = GENERATOR_DIR + '/data/patches/gpt_conut_1.txt'
generate_gpt_conut(vocab_file, model_file, input_file, identifier_txt_file, identifier_token_file, output_file, beam_size)



50061
GPTFConvModel(
  (embed_model): OpenAIGPTLMHeadModel(
    (transformer): OpenAIGPTModel(
      (tokens_embed): Embedding(50061, 384)
      (positions_embed): Embedding(1024, 384)
      (drop): Dropout(p=0, inplace=False)
      (h): ModuleList(
        (0): Block(
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0, inplace=False)
            (resid_dropout): Dropout(p=0, inplace=False)
          )
          (ln_1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (dropout): Dropout(p=0, inplace=False)
          )
          (ln_2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        )
        (1): Block(
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0, inplace=False)
            (resid_dropout): Dropout(p=0, inplac

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


1 / 39
2 / 39
3 / 39
4 / 39
5 / 39
6 / 39
7 / 39
8 / 39
9 / 39
10 / 39
11 / 39
12 / 39
13 / 39
14 / 39
15 / 39
16 / 39
17 / 39
18 / 39
19 / 39
20 / 39
21 / 39
22 / 39
23 / 39
24 / 39
25 / 39
26 / 39
27 / 39
28 / 39
29 / 39
30 / 39
31 / 39
32 / 39
33 / 39
34 / 39
35 / 39
36 / 39
37 / 39
38 / 39
50061
GPTCoNuTModel(
  (embed_model): OpenAIGPTLMHeadModel(
    (transformer): OpenAIGPTModel(
      (tokens_embed): Embedding(50061, 384)
      (positions_embed): Embedding(1024, 384)
      (drop): Dropout(p=0, inplace=False)
      (h): ModuleList(
        (0): Block(
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0, inplace=False)
            (resid_dropout): Dropout(p=0, inplace=False)
          )
          (ln_1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (dropout): Dropout(p=0, inplace=False)
          )
  

In [22]:
import codecs
import json
import os

RERANK_DIR = os.path.abspath('/content')


def read_defects4j_meta(meta_path):
    fp = codecs.open(meta_path, 'r', 'utf-8')
    meta = []
    for l in fp.readlines():
        proj, bug_id, path, start_loc, end_loc = l.strip().split('\t')
        meta.append([
            proj, bug_id, path, start_loc, end_loc + 1
        ])
    return meta


def read_quixbugs_meta(meta_path):
    fp = codecs.open(meta_path, 'r', 'utf-8')
    meta = []
    for l in fp.readlines():
        proj, loc = l.strip().split('\t')
        if '-' in loc:
            start_loc, end_loc = loc.split('-')
            start_loc = int(start_loc)
            end_loc = int(end_loc)
        else:
            start_loc, end_loc = int(loc), int(loc) + 1
        meta.append([
            proj, str(start_loc), str(end_loc)
        ])
    return meta


def read_hypo(hypo_path):
    fp = codecs.open(hypo_path, 'r', 'utf-8')
    hypo = {}
    for l in fp.readlines():
        l = l.strip().split()
        if l[0][:2] == 'S-':
            id = int(l[0][2:])
            src = ' '.join(l[1:]).strip()
            src = src.replace('@@ ', '')
            hypo[id] = {'src': src, 'patches': []}
        if l[0][:2] == 'H-':
            id = int(l[0][2:])
            patch = ' '.join(l[2:]).strip()
            patch = patch.replace('@@ ', '')
            score = float(l[1])
            hypo[id]['patches'].append([patch, score])
    return hypo


def cure_rerank(meta, hypo_path_list, output_path):
    # the patch with same rank from different models are grouped together
    group_by_rank = {}
    for hypo_path in hypo_path_list:
        hypo = read_hypo(hypo_path)
        print('finish loading', hypo_path)
        for id in hypo:
            if id not in group_by_rank:
                group_by_rank[id] = {'src': hypo[id]['src'], 'patches': []}
            for rank, (patch, score) in enumerate(hypo[id]['patches']):
                if rank >= len(group_by_rank[id]['patches']):
                    group_by_rank[id]['patches'].append([])
                group_by_rank[id]['patches'][rank].append([patch, score])

    # the patch with same rank are ranked by scores
    reranked_hypo = {}
    print('start ranking')
    for id in group_by_rank:
        key = '-'.join(meta[id])
        reranked_hypo[key] = {'src': group_by_rank[id]['src'], 'patches': []}

        added_patches = set()
        for patches_same_rank in group_by_rank[id]['patches']:
            ranked_by_score = sorted(patches_same_rank, key=lambda e: e[1], reverse=True)
            for patch, score in ranked_by_score:
                if patch not in added_patches:
                    added_patches.add(patch)
                    reranked_hypo[key]['patches'].append({'patch': patch, 'score': score})
            if len(added_patches) >= 5000:
                break

    print('dumping result in json file')
    json.dump(reranked_hypo, open(output_path, 'w'), indent=2)

In [29]:
meta_path = RERANK_DIR + '/candidate_patches/QuixBugs/meta.txt'
quixbugs_meta = read_quixbugs_meta(meta_path)
hypo_path_list = [RERANK_DIR + '/data/patches/gpt_conut_1.txt', RERANK_DIR + '/data/patches/gpt_fconv_1.txt']
output_path = RERANK_DIR + '/data/patches/reranked_patches.json'
cure_rerank(quixbugs_meta, hypo_path_list, output_path)

finish loading /content/data/patches/gpt_conut_1.txt
finish loading /content/data/patches/gpt_fconv_1.txt
start ranking
dumping result in json file


In [30]:
import json
import os
import shutil
import time
import subprocess
import sys

VALIDATE_QUIXBUGS_DIR = os.path.abspath('/content')
sys.path.append(VALIDATE_QUIXBUGS_DIR + '/src/dataloader/')

import tokenization


def command_with_timeout(cmd, timeout=5):
    p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, universal_newlines=True)
    t_beginning = time.time()
    while True:
        if p.poll() is not None:
            break
        seconds_passed = time.time() - t_beginning
        if timeout and seconds_passed > timeout:
            p.terminate()
            return 'TIMEOUT', 'TIMEOUT'
        time.sleep(1)
    out, err = p.communicate()
    return out, err


def compile_fix(filename, tmp_dir):
    FNULL = open(os.devnull, 'w')
    p = subprocess.call(["javac",
                         tmp_dir + "Node.java",
                         tmp_dir + "WeightedEdge.java",
                         filename], stderr=FNULL)
    return False if p else True


def quixbugs_test_suite(algo, quixbugs_dir):
    QUIXBUGS_MAIN_DIR = quixbugs_dir
    CUR_DIR = os.getcwd()
    FNULL = open(os.devnull, 'w')
    try:
        os.chdir(QUIXBUGS_MAIN_DIR)
        p1 = subprocess.Popen(["/usr/bin/javac", "-cp", ".:java_programs:junit4-4.12.jar:hamcrest-all-1.3.jar", 
                                "java_testcases/junit/" + algo.upper() + "_TEST.java"],
                                stdout=subprocess.PIPE, stderr=FNULL, universal_newlines=True)
        out, err = command_with_timeout(
            ["/usr/bin/java", "-cp", ".:java_programs:junit4-4.12.jar:hamcrest-all-1.3.jar",
             "org.junit.runner.JUnitCore", "java_testcases.junit." + algo.upper() + "_TEST"], timeout=5
        )

        os.chdir(CUR_DIR)
        if "FAILURES" in str(out) or "FAILURES" in str(err):
            return 'wrong'
        elif "TIMEOUT" in str(out) or "TIMEOUT" in str(err):
            return 'timeout'
        else:
            return 'plausible'
    except Exception as e:
        print(e)
        os.chdir(CUR_DIR)
        return 'uncompilable'


def insert_fix_quixbugs(file_path, start_loc, end_loc, patch):
    shutil.copyfile(file_path, file_path + '.bak')

    with open(file_path, 'r') as file:
        data = file.readlines()

    patched = False
    with open(file_path, 'w') as file:
        for idx, line in enumerate(data):
            if start_loc - 1 <= idx < end_loc - 1:
                if not patched:
                    file.write(patch)
                    patched = True
            else:
                file.write(line)

    return file_path + '.bak'


def get_strings_numbers(file_path, loc):
    numbers_set = {}
    strings_set = {}
    with open(file_path, 'r') as file:
        data = file.readlines()
        for idx, line in enumerate(data):
            dist = loc - idx - 1
            strings, numbers = tokenization.get_strings_numbers(line)
            for num in numbers:
                if num != '0' and num != '1':
                    if num in numbers_set:
                        numbers_set[num] = min(dist, numbers_set[num])
                    else:
                        numbers_set[num] = dist
            for str in strings:
                if str in strings_set:
                    strings_set[str] = min(dist, strings_set[str])
                else:
                    strings_set[str] = dist
    final_strings = []
    final_numbers = []
    for k, v in numbers_set.items():
        final_numbers.append([k, v])
    for k, v in strings_set.items():
        final_strings.append([k, v])
    final_numbers.sort(key=lambda x: x[1])
    final_strings.sort(key=lambda x: x[1])
    return final_strings, final_numbers


cnt, right = 0, 0


def validate_quixbugs(reranked_result_path, output_path, tmp_dir):
    global cnt, right

    if not os.path.exists(tmp_dir):
        command_with_timeout(['mkdir', tmp_dir])

    reranked_result = json.load(open(reranked_result_path, 'r'))
    validated_result = {}
    for key in reranked_result:
        cnt += 1

        proj, start_loc, end_loc = key.split('-')
        print(right, '/', cnt, proj)

        command_with_timeout(['rm', '-rf', tmp_dir + '/java_programs/'])
        command_with_timeout(['mkdir', tmp_dir + '/java_programs/'])

        shutil.copyfile(tmp_dir + "/java_programs_bak/" + proj + '.java',
                        tmp_dir + "/java_programs/" + proj + '.java')
        shutil.copyfile(tmp_dir + "/java_programs_bak/Node.java", tmp_dir + "/java_programs/Node.java")
        shutil.copyfile(tmp_dir + "/java_programs_bak/WeightedEdge.java", tmp_dir + "/java_programs/WeightedEdge.java")

        validated_result[key] = {'src': reranked_result[key]['src'], 'patches': []}
        bug_start_time = time.time()
        current_is_correct = False
        for tokenized_patch in reranked_result[key]['patches']:
            # validate 5 hours for each bug at most
            if time.time() - bug_start_time > 5 * 3600:
                break
            # validate 5000 patches for each bug at most
            if len(validated_result[key]['patches']) >= 5000:
                break
            filename = tmp_dir + "/java_programs/" + proj + '.java'

            score = tokenized_patch['score']
            tokenized_patch = tokenized_patch['patch']

            strings, numbers = get_strings_numbers(filename, (int(start_loc) + int(end_loc)) // 2)
            strings = [item[0] for item in strings][:5]
            numbers = [item[0] for item in numbers][:5]
            # one tokenized patch may be reconstructed to multiple source-code patches
            reconstructed_patches = tokenization.token2statement(tokenized_patch.split(' '), numbers, strings)
            # validate most 5 source-code patches come from the same tokenized patch
            for patch in reconstructed_patches[:5]:
                patch = patch.strip()
                insert_fix_quixbugs(filename, int(start_loc), int(end_loc), patch)
                compile = compile_fix(filename, tmp_dir + "/java_programs/")
                correctness = 'uncompilable'
                if compile:
                    correctness = quixbugs_test_suite(proj, quixbugs_dir=tmp_dir)
                    if correctness == 'plausible':
                        if not current_is_correct:
                            right += 1
                            current_is_correct = True
                        print(right, '/', cnt, "Plausible patch:", patch)
                        break
                    elif correctness == 'wrong':
                        print(right, '/', cnt, "Wrong patch:", patch)
                    elif correctness == 'timeout':
                        print(right, '/', cnt, "Timeout patch:", patch)
                else:
                    print(right, '/', cnt, 'Uncompilable patch:', patch)
                validated_result[key]['patches'].append({
                    'patch': patch, 'correctness': correctness
                })
                shutil.copyfile(tmp_dir + "/java_programs_bak/" + proj + '.java',
                                tmp_dir + "/java_programs/" + proj + '.java')
            json.dump(validated_result, open(output_path, 'w'), indent=2)
            if current_is_correct:
                break
        json.dump(validated_result, open(output_path, 'w'), indent=2)


In [37]:
    reranked_result_path = VALIDATE_QUIXBUGS_DIR + '/data/patches/reranked_patches.json'
    output_path = VALIDATE_QUIXBUGS_DIR + '/data/patches/validated_patches.json'
    tmp_dir = VALIDATE_QUIXBUGS_DIR + '/tmp/validate_quixbugs'
    validate_quixbugs(reranked_result_path, output_path, tmp_dir)

0 / 6 BITCOUNT


FileNotFoundError: ignored